In [ ]:
path = ''

# 환경설정

In [5]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import matplotlib.pyplot as plt
import openai
from openai import OpenAI
import json





In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from warnings import filterwarnings
FutureWarning
filterwarnings('ignore')

## 환경 변수 설정 ( API KEY 설정)

In [ ]:
def load_file(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

# API 키 로드 및 환경변수 설정
openai.api_key = load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

In [ ]:
print(os.environ['OPENAI_API_KEY'])

# 음성

# 텍스트

In [ ]:
data = pd.read_csv(path + '/')
data.head()

#### 데이터 준비

In [ ]:
# 데이터셋 분할
train, val = train_test_split(data, test_size=0.2, random_state=42)
# 텐서 데이터셋으로 변환
train_ds = Dataset.from_pandas(train)
val_ds = Dataset.from_pandas(val)

#### (토큰화)

In [ ]:
# 모델과 토크나이저 불러오기
# 모델과 토크나이저 불러오기
model_name = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# 토큰화 함수 생성 및 작업
def preprocess_function(data):
    return tokenizer(data['text'], truncation=True, padding=True)

train_ds = train_ds.map(preprocess_function, batched=True)
val_ds = val_ds.map(preprocess_function, batched=True)

#### 데이터 생성 (음성 추출)

In [25]:
import requests
import json
from pytube import YouTube
from pydub import AudioSegment

# Step 1: 유튜브 오디오 다운로드
def download_youtube_audio(video_url, output_path="audio/audio.wav"):
    # yt = YouTube(video_url)
    yt = YouTube(video_url, use_oauth=True, allow_oauth_cache=True)

    audio_stream = yt.streams.filter(only_audio=True).first()
    temp_audio_path = "audio/temp_audio.mp4"
    audio_stream.download(output_path="audio/", filename="temp_audio.mp4")
    
    # MP4 -> WAV 변환
    audio = AudioSegment.from_file(temp_audio_path, format="mp4")
    audio.export(output_path, format="wav")
    print("오디오 다운로드 및 변환 완료!")
    return output_path

# Step 2: 클로바 API를 사용한 음성 텍스트 변환
def audio_to_text_clova(audio_path, api_url, client_id, client_secret):
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/octet-stream"
    }

    with open(audio_path, "rb") as audio_file:
        response = requests.post(api_url, headers=headers, data=audio_file)
    
    if response.status_code == 200:
        result = response.json()
        return result.get("text", "")  # 변환된 텍스트 반환
    else:
        print(f"오류 발생: {response.status_code}, {response.text}")
        return None

# Step 3: 텍스트 저장
def save_text_to_file(text, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"텍스트가 {file_path}에 저장되었습니다!")



In [52]:
import os
import yt_dlp

def download_youtube_audio(video_url):
    try:
        # 지정된 절대 경로
        output_path = r"C:\Users\User\Desktop\eduSmile-master\audio"
        
        # 출력 디렉토리 생성
        os.makedirs(output_path, exist_ok=True)
        
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
            }],
            'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
            'ffmpeg_location': 'C:\\ffmpeg\\bin'  # ffmpeg 경로
        }
        
        print("다운로드 시작...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([video_url])
        print(f"다운로드 완료! 저장 위치: {output_path}")
        
    except Exception as e:
        print(f"Error: {str(e)}")



In [55]:
def process_youtube_urls():
    """
    txt 파일에서 YouTube URL을 읽어서 오디오를 다운로드하는 함수
    """
    # 파일 경로 설정
    file_path = r"C:\Users\User\Desktop\eduSmile-master\.git\datasets\youtube_datasets_01.txt"
    output_path = r"C:\Users\User\Desktop\eduSmile-master\audio"
    
    try:
        # 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            # https로 시작하는 URL만 필터링
            urls = [line.strip() for line in file if line.strip().startswith('https')]
        
        print(f"총 {len(urls)}개의 URL을 찾았습니다.")
        
        # 각 URL에 대해 오디오 다운로드
        for idx, url in enumerate(urls, 1):
            print(f"\n[{idx}/{len(urls)}] URL 처리 중...")
            download_youtube_audio(url)
            
    except FileNotFoundError:
        print(f"Error: 파일을 찾을 수 없습니다: {file_path}")
    except Exception as e:
        print(f"Error: 파일 처리 중 오류가 발생했습니다: {str(e)}")

In [56]:
process_youtube_urls()

Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


총 20개의 URL을 찾았습니다.

[1/20] URL 처리 중...
다운로드 시작...
[youtube] Extracting URL: https://www.youtube.com/watch?v=zlaMDt04gXY
[youtube] zlaMDt04gXY: Downloading webpage
[youtube] zlaMDt04gXY: Downloading ios player API JSON
[youtube] zlaMDt04gXY: Downloading mweb player API JSON


ERROR: [youtube] zlaMDt04gXY: Video unavailable
Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


Error: ERROR: [youtube] zlaMDt04gXY: Video unavailable

[2/20] URL 처리 중...
다운로드 시작...
[youtube] Extracting URL: https://www.youtube.com/watch?v=xBnq7xf2wt8
[youtube] xBnq7xf2wt8: Downloading webpage
[youtube] xBnq7xf2wt8: Downloading ios player API JSON
[youtube] xBnq7xf2wt8: Downloading mweb player API JSON
[youtube] xBnq7xf2wt8: Downloading player a5032d18


         player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js
         n = WDsROWxyLpwasOWE ; player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js
         player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js
         n = RJKVHZKVCvX-oGpA ; player = https://www.youtube.com/s/player/a5032d18/player_ias.vflset/en_US/base.js


...

[youtube] E7YyNgT-qaY: Downloading m3u8 information
[info] E7YyNgT-qaY: Downloading 1 format(s): 251
[download] Destination: C:\Users\User\Desktop\eduSmile-master\audio\정수 사칙연산.webm
[download] 100% of    7.99MiB in 00:00:02 at 3.45MiB/s   
[ExtractAudio] Destination: C:\Users\User\Desktop\eduSmile-master\audio\정수 사칙연산.wav
Deleting original file C:\Users\User\Desktop\eduSmile-master\audio\정수 사칙연산.webm (pass -k to keep)


Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


다운로드 완료! 저장 위치: C:\Users\User\Desktop\eduSmile-master\audio

[20/20] URL 처리 중...
다운로드 시작...
[youtube] Extracting URL: https://www.youtube.com/watch?v=GnaQolCOvZw
[youtube] GnaQolCOvZw: Downloading webpage
[youtube] GnaQolCOvZw: Downloading ios player API JSON
[youtube] GnaQolCOvZw: Downloading mweb player API JSON


ERROR: [youtube] GnaQolCOvZw: Video unavailable


Error: ERROR: [youtube] GnaQolCOvZw: Video unavailable


#### 음성에서 텍스트로 변환

In [60]:
import requests
import json
import base64

class ClovaSpeechClient:
    """
    네이버 클로바 Speech-to-Text API 클라이언트
    """
    def __init__(self, client_id, client_secret):
        self.api_url = "https://naveropenapi.apigw.ntruss.com/recog/v1/stt"
        self.client_id = client_id
        self.client_secret = client_secret

    def speech_to_text(self, audio_file, lang='Kor'):
        """
        음성 파일을 텍스트로 변환
        
        Parameters:
        - audio_file (str): 음성 파일 경로 (.wav, .mp3, .opus 등)
        - lang (str): 언어 설정 (Kor, Eng, Jpn, Chn)
        
        Returns:
        - str: 변환된 텍스트
        """
        headers = {
            "X-NCP-APIGW-API-KEY-ID": self.client_id,
            "X-NCP-APIGW-API-KEY": self.client_secret,
            "Content-Type": "application/octet-stream"
        }
        
        params = {
            "lang": lang,
            "completion": "sync"  # 동기 호출
        }
        
        try:
            # 음성 파일 읽기 및 크기 확인
            with open(audio_file, 'rb') as f:
                audio_data = f.read()
                
            # 파일 크기 확인 (최대 허용 크기: 300MB)
            file_size_mb = len(audio_data) / (1024 * 1024)
            if file_size_mb > 300:
                raise Exception(f"파일 크기가 너무 큽니다: {file_size_mb:.2f}MB (최대 300MB)")
            
            print(f"파일 크기: {file_size_mb:.2f}MB")
            
            # API 요청
            response = requests.post(
                self.api_url,
                headers=headers,
                params=params,
                data=audio_data
            )
            
            # 응답 상태 확인 및 디버깅
            print(f"Response Status Code: {response.status_code}")
            print(f"Response Headers: {response.headers}")
            
            try:
                result = response.json()
                print(f"Response Content: {result}")
            except json.JSONDecodeError:
                print(f"Raw Response Content: {response.text}")
                raise Exception("API 응답을 JSON으로 파싱할 수 없습니다.")
            
            if response.status_code == 200:
                if 'text' not in result:
                    raise Exception("API 응답에 텍스트 결과가 없습니다.")
                return result.get('text', '')
            else:
                error_message = result.get('errorMessage', '')
                error_code = result.get('errorCode', '')
                raise Exception(f"API 호출 실패 - Status: {response.status_code}, Code: {error_code}, Message: {error_message}")
                
        except requests.RequestException as e:
            raise Exception(f"API 요청 중 오류 발생: {str(e)}")
        except Exception as e:
            raise Exception(f"음성 변환 중 오류 발생: {str(e)}")

# 사용 예시


#### 클로바 버전


In [61]:

import os
from pathlib import Path

# API 인증 정보 설정
# 사용 예시    # API 인증 정보 설정
CLIENT_ID = "8wx5qqkgjj"
CLIENT_SECRET = "XVa7Xy8aJwxRaWLeuq72VQzwe668mbIRcYTCwEfX"

# 경로 설정
audio_dir = r"C:\Users\User\Desktop\eduSmile-master\audio"
text_dir = r"C:\Users\User\Desktop\eduSmile-master\text"

# 클라이언트 초기화
stt_client = ClovaSpeechClient(CLIENT_ID, CLIENT_SECRET)

try:
    # wav 파일 목록 가져오기
    wav_files = sorted([f for f in os.listdir(audio_dir) if f.endswith('.wav')])
    
    # 각 wav 파일 처리
    for i, wav_file in enumerate(wav_files, 1):
        wav_path = os.path.join(audio_dir, wav_file)
        
        try:
            # 파일 존재 확인
            if not os.path.exists(wav_path):
                print(f"파일을 찾을 수 없음: {wav_file}")
                continue
                
            # 음성을 텍스트로 변환
            print(f"\n처리 중: {wav_file}")
            print(f"파일 경로: {wav_path}")
            
            result = stt_client.speech_to_text(
                audio_file=wav_path,
                lang="Kor"  # 한국어 음성 인식
            )
            
            # 결과를 텍스트 파일로 저장
            text_filename = f"{i:03d}.txt"  # 001.txt, 002.txt 형식
            text_path = os.path.join(text_dir, text_filename)
            
            with open(text_path, 'w', encoding='utf-8') as f:
                f.write(result)
            
            print(f"변환 성공 - 저장 완료: {text_filename}")
            print(f"변환된 텍스트: {result[:100]}...")  # 처음 100자만 출력
            
        except Exception as e:
            print(f"\n파일 처리 실패: {wav_file}")
            print(f"오류 내용: {str(e)}")
            continue  # 다음 파일 처리 계속
        
except Exception as e:
    print(f"오류 발생: {str(e)}")


처리 중: 1.2 정수와 유리수 (4) 사칙연산 Part1 덧셈편1 ｜ 중1수학 1학기.wav
파일 경로: C:\Users\User\Desktop\eduSmile-master\audio\1.2 정수와 유리수 (4) 사칙연산 Part1 덧셈편1 ｜ 중1수학 1학기.wav
파일 크기: 145.31MB
Response Status Code: 413
Response Headers: {'Server': 'nginx', 'Date': 'Thu, 09 Jan 2025 05:45:10 GMT', 'Content-Type': 'application/json', 'Content-Length': '66', 'Connection': 'close'}
Response Content: {'error': {'errorCode': '430', 'message': 'Request Entity Too Large'}}

파일 처리 실패: 1.2 정수와 유리수 (4) 사칙연산 Part1 덧셈편1 ｜ 중1수학 1학기.wav
오류 내용: 음성 변환 중 오류 발생: API 호출 실패 - Status: 413, Code: , Message: 

처리 중: Simplemaths의 수학 강의 - 방정식 - 음수의 사칙 연산 -2편-.wav
파일 경로: C:\Users\User\Desktop\eduSmile-master\audio\Simplemaths의 수학 강의 - 방정식 - 음수의 사칙 연산 -2편-.wav
파일 크기: 102.14MB
Response Status Code: 413
Response Headers: {'Server': 'nginx', 'Date': 'Thu, 09 Jan 2025 05:45:30 GMT', 'Content-Type': 'application/json', 'Content-Length': '66', 'Connection': 'close'}
Response Content: {'error': {'errorCode': '430', 'message': 'Request Entity To

#### Whisper |

In [63]:
import os
from pathlib import Path
import openai
from openai import OpenAI
import time

class WhisperTranscriber:
    def __init__(self, api_key):
        """
        OpenAI Whisper API 초기화
        
        Parameters:
        - api_key (str): OpenAI API 키
        """
        self.client = OpenAI(api_key=api_key)

    def transcribe_audio(self, audio_path):
        """
        음성 파일을 텍스트로 변환
        
        Parameters:
        - audio_path (str): 음성 파일 경로
        
        Returns:
        - str: 변환된 텍스트
        """
        try:
            print(f"\nProcessing: {os.path.basename(audio_path)}")
            
            # 파일 열기
            with open(audio_path, "rb") as audio_file:
                # Whisper API 호출
                result = self.client.audio.transcriptions.create(
                    model="whisper-1",
                    file=audio_file,
                    language="ko",  # 한국어
                    response_format="text"
                )
            
            return result
            
        except Exception as e:
            raise Exception(f"Transcription failed: {str(e)}")

def process_directory(audio_dir, output_dir, api_key):
    """
    디렉토리 내의 모든 음성 파일 처리
    
    Parameters:
    - audio_dir (str): 음성 파일이 있는 디렉토리 경로
    - output_dir (str): 결과를 저장할 디렉토리 경로
    - api_key (str): OpenAI API 키
    """
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # Whisper 초기화
    transcriber = WhisperTranscriber(api_key)
    
    # 음성 파일 목록 가져오기 (지원 형식: m4a, mp3, mp4, mpeg, mpga, wav, webm)
    audio_files = sorted([
        f for f in os.listdir(audio_dir)
        if f.lower().endswith(('.m4a', '.mp3', '.mp4', '.mpeg', '.mpga', '.wav', '.webm'))
    ])
    
    print(f"\nFound {len(audio_files)} audio files to process")
    
    # 각 파일 처리
    for i, audio_file in enumerate(audio_files, 1):
        try:
            print(f"\n=== Processing file {i}/{len(audio_files)} ===")
            print(f"File: {audio_file}")
            
            audio_path = os.path.join(audio_dir, audio_file)
            
            # 파일 크기 확인 (API 제한: 25MB)
            file_size_mb = os.path.getsize(audio_path) / (1024 * 1024)
            if file_size_mb > 25:
                print(f"File size ({file_size_mb:.2f}MB) exceeds 25MB limit, skipping...")
                continue
            
            # 음성을 텍스트로 변환
            text = transcriber.transcribe_audio(audio_path)
            
            # 결과를 파일로 저장
            output_file = f"{i:03d}.txt"
            output_path = os.path.join(output_dir, output_file)
            
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(text)
            
            print(f"Successfully saved to: {output_file}")
            print(f"Transcribed text (preview): {text[:200]}...")
            
            # API 요청 제한을 위한 대기
            if i < len(audio_files):
                print("Waiting 2 seconds before next file...")
                time.sleep(2)
            
        except Exception as e:
            print(f"\nError processing {audio_file}")
            print(f"Error details: {str(e)}")
            continue



TypeError: argument of type 'NoneType' is not iterable

In [ ]:
# 경로 설정
audio_dir = r"C:\Users\User\Desktop\eduSmile-master\audio"
text_dir = r"C:\Users\User\Desktop\eduSmile-master\text"
# OpenAI API 키 설정
api_key = "your-api-key-here"  # OpenAI API 키를 여기에 입력하세요

try:
    # 디렉토리 내 모든 파일 처리
    process_directory(audio_dir, text_dir, api_key)
    print("\nProcessing completed successfully!")
    
except Exception as e:
    print(f"\nAn error occurred: {str(e)}")

#### 모델 학습

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import numpy as np

class LectureDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

def train_classifier(train_texts, train_labels, model_name='bert-base-multilingual-cased'):
    # 토크나이저와 모델 초기화
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2  # 수업내용(0)과 공지사항(1)
    )
    
    # 데이터셋 생성
    dataset = LectureDataset(train_texts, train_labels, tokenizer)
    train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
    
    # 학습 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    
    # 학습 루프
    model.train()
    for epoch in range(3):  # 3 에폭 동안 학습
        for batch in train_loader:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            loss.backward()
            optimizer.step()
    
    return model, tokenizer

def classify_notes(text, model, tokenizer, device='cuda'):
    """
    텍스트를 수업내용(0)과 공지사항(1)으로 분류
    """
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
    
    probabilities = torch.softmax(outputs.logits, dim=1)
    prediction = torch.argmax(probabilities, dim=1).item()
    
    return {
        'class': 'notice' if prediction == 1 else 'lecture',
        'probability': probabilities[0][prediction].item()
    }

# 사용 예시



In [ ]:
import openai
from typing import Dict, List
import os

class LectureNoteClassifier:
    def __init__(self, api_key: str):
        """
        OpenAI API를 사용하여 텍스트를 수업 내용, 공지사항, 관련없는 내용으로 분류하는 클래스
        
        Args:
            api_key (str): OpenAI API 키
        """
        openai.api_key = api_key
        
        # 분류를 위한 프롬프트 템플릿
        self.prompt_template = """다음 텍스트를 아래 세 카테고리 중 하나로 분류해주세요:
- lecture: 수업 내용 (예: 이론 설명, 개념 설명 등)
- notice: 수업 관련 공지사항 (예: 시험 일정, 과제 제출, 휴강 공지 등)
- unrelated: 수업과 관련없는 내용 (예: 개인적인 메모, 잡담, 다른 과목 내용 등)

답변은 위의 세 단어 중 하나만 작성해주세요.

텍스트: {text}

분류:"""

    def classify_line(self, text: str) -> Dict:
        """
        한 줄의 텍스트를 분류합니다.
        
        Args:
            text (str): 분류할 텍스트
            
        Returns:
            Dict: 분류 결과와 신뢰도를 포함한 딕셔너리
        """
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",  # 또는 "gpt-3.5-turbo"
                messages=[
                    {"role": "system", "content": "당신은 수업 노트를 수업 내용, 공지사항, 관련없는 내용으로 분류하는 도우미입니다."},
                    {"role": "user", "content": self.prompt_template.format(text=text)}
                ],
                temperature=0.3
            )
            
            # API 응답에서 분류 결과 추출
            classification = response.choices[0].message.content.strip().lower()
            
            return {
                'text': text,
                'class': classification,
                'probability': 1.0
            }
            
        except Exception as e:
            print(f"Error during classification: {e}")
            return {
                'text': text,
                'class': 'unknown',
                'probability': 0.0
            }

    def classify_notes(self, notes_text: str) -> Dict[str, List]:
        """
        여러 줄의 텍스트를 분류합니다.
        
        Args:
            notes_text (str): 분류할 전체 텍스트
            
        Returns:
            Dict: 수업 내용, 공지사항, 관련없는 내용으로 분류된 결과
        """
        # 줄바꿈을 기준으로 텍스트를 분리하고 빈 줄 제거
        lines = [line.strip() for line in notes_text.split('\n') if line.strip()]
        
        results = {
            'lecture_content': [],
            'notices': [],
            'unrelated_content': []
        }
        
        # 각 줄 분류
        for line in lines:
            classification = self.classify_line(line)
            if classification['class'] == 'lecture':
                results['lecture_content'].append({
                    'text': line,
                    'probability': classification['probability']
                })
            elif classification['class'] == 'notice':
                results['notices'].append({
                    'text': line,
                    'probability': classification['probability']
                })
            elif classification['class'] == 'unrelated':
                results['unrelated_content'].append({
                    'text': line,
                    'probability': classification['probability']
                })
        
        return results

# 사용 예시
def main():
    # OpenAI API 키 설정
    api_key = "your_openai_api_key_here"  # 실제 API 키로 교체 필요
    
    # 분류기 초기화
    classifier = LectureNoteClassifier(api_key)
    
    # 테스트용 노트
    test_notes = """오늘은 선형대수학의 기초를 배우겠습니다.
행렬의 곱셈은 다음과 같이 정의됩니다.
다음 주 퀴즈가 있으니 준비해주세요.
점심 메뉴 추천: 학식 오늘 맛있대.
벡터의 내적은 두 벡터의 성분끼리 곱한 것의 합입니다.
과제는 이번 주 금요일까지 제출해주세요.
오늘 날씨 진짜 덥다...
행렬식의 기하학적 의미는 면적 또는 부피를 나타냅니다.
저녁에 친구랑 약속 있음"""

    # 노트 분류
    results = classifier.classify_notes(test_notes)
    
    # 결과 출력
    print("\n=== 수업 내용 ===")
    for item in results['lecture_content']:
        print(f"• {item['text']}")
    
    print("\n=== 공지사항 ===")
    for item in results['notices']:
        print(f"• {item['text']}")
        
    print("\n=== 수업 무관 내용 ===")
    for item in results['unrelated_content']:
        print(f"• {item['text']}")

if __name__ == "__main__":
    main()

In [ ]:

# 학습 데이터 예시
train_texts = [
    "다음 주 시험은 3장부터 5장까지입니다.",  # 공지사항
    "오늘은 미분방정식의 기초에 대해 배우겠습니다.",  # 수업내용
    "과제 제출 기한은 이번 주 금요일까지입니다.",  # 공지사항
    "함수의 극한값을 구하는 방법은 다음과 같습니다."  # 수업내용
]
train_labels = [1, 0, 1, 0]  # 1: 공지사항, 0: 수업내용

# 모델 학습
model, tokenizer = train_classifier(train_texts, train_labels)

# 새로운 텍스트 분류
test_text = "다음 주는 중간고사 기간이므로 수업이 없습니다."
result = classify_notes(test_text, model, tokenizer)
print(f"분류 결과: {result['class']}, 확률: {result['probability']:.2f}")